# Brewery Recommender

In [1]:
# importing libraries
import pandas as pd
import sys
from scipy import sparse
from sklearn.metrics.pairwise import pairwise_distances
import matplotlib.pyplot as plt

from tqdm import tqdm
import numpy as np
import requests
from bs4 import BeautifulSoup

In [2]:
# reading in full data set for breweries
df_full = pd.read_csv('../data/beer_reviews_no_ones.csv')
df_full.head()

,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
0,1075,Caldera Brewing Company,2012-01-02 17:17:39,3.0,3.5,3.5,oline73,Herbed / Spiced Beer,3.0,3.5,Caldera Ginger Beer,4.7,52159
1,1075,Caldera Brewing Company,2011-10-19 02:25:15,3.5,3.5,3.5,Reidrover,Herbed / Spiced Beer,4.0,4.0,Caldera Ginger Beer,4.7,52159
2,1075,Caldera Brewing Company,2011-05-24 22:26:58,3.0,2.5,3.5,alpinebryant,Herbed / Spiced Beer,2.0,3.5,Caldera Ginger Beer,4.7,52159
3,1075,Caldera Brewing Company,2010-11-22 19:35:03,4.0,3.0,3.5,LordAdmNelson,Herbed / Spiced Beer,3.5,4.0,Caldera Ginger Beer,4.7,52159
4,1075,Caldera Brewing Company,2010-09-28 00:15:24,4.5,3.5,5.0,augustgarage,Herbed / Spiced Beer,4.0,4.0,Caldera Ginger Beer,4.7,52159


In [3]:
# reading in data
df = pd.read_csv('../data/beer_reviews_col_drop.csv')
df.head()

,review_overall,review_profilename,beer_name
0,3.0,oline73,Caldera Ginger Beer
1,3.5,Reidrover,Caldera Ginger Beer
2,3.0,alpinebryant,Caldera Ginger Beer
3,4.0,LordAdmNelson,Caldera Ginger Beer
4,4.5,augustgarage,Caldera Ginger Beer


In [4]:
# df_sample = df.sample(frac=0.1, random_state=1) maybe a good way to cut down on data

In [5]:
pivot = df.pivot_table(index='beer_name', columns='review_profilename', 
                       values=['review_overall'])

In [7]:
pivot_sparse = sparse.csr_matrix(pivot.fillna(0))

In [8]:
recommender = pairwise_distances(pivot_sparse, metric='cosine')

In [9]:
recommender_df = pd.DataFrame(recommender, index=pivot.index, columns=pivot.index)

In [10]:
pivot = []
recommender = []
pivot_sparse = []

In [20]:
recommender_df.shape

(37897, 37897)

In [23]:
recommender_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 37897 entries, "100" to Ω-naught (Omeganaught)
Columns: 37897 entries, "100" to Ω-naught (Omeganaught)
dtypes: float64(37897)
memory usage: 10.7+ GB


In [22]:
# short_recommender = recommender_df[10_000]

KeyError: 10000

In [11]:
recommender_df['\"400\" Ale'].sort_values()[1:10]

beer_name
Milly's Labyrinth                          0.636757
Milly's Lil' Ivan's Foreign Extra Stout    0.683922
Portsmouth Junebock                        0.688553
India Special Reserve Beer                 0.695276
Guinness Foreign Extra Stout (Jamaica)     0.705559
Milly's Hopzilla                           0.706863
Olde Richmond Batch No. 9 Red Ale          0.709080
Weyermann Imperial India Pale Ale          0.716571
Moat Golden Dog Pilsner                    0.728785
Name: "400" Ale, dtype: float64

In [37]:
recs = []
for key, value in user_1_rating.items():
    rec = {}
    if value >= 4: # could change this for lower rated beers
        rec[key] = recommender_df[key].sort_values()[1:25] # don't think I need this limiter because the <= 0.5 is doing that for me later on
        recs.append(rec)

beer_checklist = []
for i in recs:
    for beer, scores in i.items():
        for recom, sim_scores in scores.items():
            if sim_scores <= 0.5: # maybe I shouldn't throw away anything...
                beer_checklist.append((recom, sim_scores))

breweries = []
current_beer = ''
for beer in beer_checklist:
    for index, brewery in df_full[df_full['beer_name'] == beer[0]]['brewery_name'].items():
        if current_beer != beer[0]:
            current_beer = beer[0] 
            brewery_score = (brewery, beer[1])
            breweries.append(brewery_score)

current_brewery = ''
scores = []
brewery_dict = {}
for i in range(len(sorted(breweries))):
    if sorted(breweries)[i][0] == current_brewery:
        scores.append(sorted(breweries)[i][1])
    else:
        if current_brewery:
            brewery_dict[current_brewery] = scores
        current_brewery = sorted(breweries)[i][0]
        scores = []
        scores.append(sorted(breweries)[i][1])

In [38]:
brewery_dict

{'5 Rabbit Cerveceria': [0.37316538389978837, 0.4347176452157713],
 '7 Seas Brewery and Taproom': [0.4547247457653536],
 'Abashiri, K.K.': [0.004106793532296038, 0.004106793532296038],
 'Anacortes Brewery & Rockfish Grill': [0.34693752501120134],
 "Artisan's Brewery & Italian Grill": [0.42460354443124937,
  0.46784791580980856],
 'Asahi Breweries Ltd': [0.4343145750507619],
 "Big Dog's Draft House": [0.41165159458544787],
 'Black Raven Brewing Co.': [0.4517288969817854],
 'Boundary Bay Brewery & Bistro': [0.42619541594696886],
 'Brouwerij Van Honsebrouck N.V.': [0.22113218638202314],
 'Budge Brothers Brewery': [0.41576260532782283],
 'Burnside Brewing Co.': [0.30871416462168844],
 'Cervezas Moritz, S.A.': [0.49412274752247887],
 'City Grille and Brewhaus': [0.33791528911810553],
 'Dunedin Brewery': [0.46312450780684067],
 'Elysian Brewing Company': [0.43837280160829284],
 'Ezo Beer KK': [0.38187746223089936],
 'Five Seasons Westside': [0.5],
 'Flossmoor Station Restaurant & Brewery': [

In [24]:
recs = []
for key, value in user_1_rating.items():
    rec = {}
    if value >= 4:
        rec[key] = recommender_df[key].sort_values()[1:11]
        recs.append(rec)

In [83]:
def brewery_recommender(user):
    recs = []
    for key, value in user.items():
        rec = {}
        if value >= 4:
            rec[key] = recommender_df[key].sort_values()[1:11]
            recs.append(rec)

    beer_checklist = []
    for i in recs:
        for beer, scores in i.items():
    #         print(beer)
    #         print()
            for recom, sim_scores in scores.items():
                if sim_scores <= 0.5:
    #                 print(recom, ': ', sim_scores)
                    beer_checklist.append((recom, sim_scores))
    #         print()
    #         print('------------------------------------------------------------------------------------------------------------------------------------------------------')

    breweries = []
    # brewery_score = {}
    brewery_check = ''
    current_beer = ''
    for beer in beer_checklist:
    #     print(beer)
        for index, brewery in df_full[df_full['beer_name'] == beer[0]]['brewery_name'].items():
    #         print('brewery: ', brewery, '          score: ', beer[1])
            if current_beer != beer[0]:
    # #             score[brewery] = beer[1]
                current_beer = beer[0]
    #             brewery_score[brewery] += beer[1]
#                 print('brewery: ', brewery)
#                 print('score: ', beer[1])"
    #         else:
    #             continue
                brewery_score = (brewery, beer[1])
                breweries.append(brewery_score)
    # breweries = set(breweries)       

    current_brewery = ''
    scores = []
    brewery_dict = {}
    for i in range(len(sorted(breweries))):
        if breweries[i][0] == current_brewery:
            scores.append(breweries[i][1])
        else:
            if current_brewery:
                brewery_dict[current_brewery] = scores
            current_brewery = breweries[i][0]
            scores = []
            scores.append(breweries[i][1])
    return brewery_dict

In [84]:
brewery_recommender(user_1_rating)

{'Abashiri, K.K.': [0.004106793532296038, 0.004106793532296038],
 'Wind Valley Brewery / Rakuno Okoku Kabushiki Kaisha Brewery': [0.21741441912877035],
 'Brouwerij Van Honsebrouck N.V.': [0.22113218638202314],
 'Kamakura Beer Brewing Company': [0.2440710539815456],
 'Schlossbrauerei Reckendorf Georg Dirauf KG': [0.2928932188134524],
 "Martha's Exchange": [0.2928932188134524],
 'Maplewood Farms Restaurant & Brewery': [0.2928932188134524],
 'City Grille and Brewhaus': [0.33791528911810553],
 'Rock Bottom Restaurant & Brewery': [0.34346783570138706],
 'Suntory': [0.3500663163803185],
 'Kirin Brewery Company, Limited': [0.35300336077936945, 0.3675444679663241],
 'Kiuchi Brewery': [0.3675444679663241],
 'Ezo Beer KK': [0.38187746223089936],
 'Shikoku Kakou K.K.': [0.3860593864850794],
 "Milly's Tavern": [0.3860593864850794],
 'Ishikari Banya No Ji-Beer': [0.41165159458544787],
 'The Tap Brewing Company': [0.41165159458544787],
 "Big Dog's Draft House": [0.41165159458544787],
 'Orion Beer Co

In [49]:
def brewery_recommender(user):
    recs = []
    for key, value in user.items():
        rec = {}
        if value >= 4: # could change this for lower rated beers
            rec[key] = recommender_df[key].sort_values()[1:] # don't think I need this limiter because the <= 0.5 is doing that for me later on
            recs.append(rec)

    beer_checklist = []
    for i in recs:
        for beer, scores in i.items():
            for recom, sim_scores in scores.items():
                if sim_scores <= 0.5: # maybe I shouldn't throw away anything...I NEED THIS TO SPEED UP!!!
                    beer_checklist.append((recom, sim_scores))
 
    breweries = []
    current_beer = ''
    for beer in beer_checklist:
        for index, brewery in df_full[df_full['beer_name'] == beer[0]]['brewery_name'].items():
            if current_beer != beer[0]:
                current_beer = beer[0] 
                brewery_score = (brewery, beer[1])
                breweries.append(brewery_score) 
                
    current_brewery = ''
    scores = []
    brewery_dict = {}
    breweries_sort = sorted(breweries)
    for i in range(len(breweries_sort)):
        if breweries_sort[i][0] == current_brewery:
            scores.append(breweries_sort[i][1])
        else:
            if current_brewery:
                brewery_dict[current_brewery] = scores
            current_brewery = breweries_sort[i][0]
            scores = []
            scores.append(breweries_sort[i][1])
    return brewery_dict

def random_beer_sample(num_beers):
    beers = df['beer_name'].sample(n=num_beers, random_state=42) # not sure if I want to set a random state here...
    np.random.seed(6) # again not sure about the randomness...
    ratings = []
    user_ratings = {}
    for i in beers:
        user_ratings[i] = np.random.uniform(high=5.0)
    return user_ratings

def sorting_brewery_scores(n_beers):
    rand_user = random_beer_sample(n_beers)
    best_breweries = {}
    for brewery, score in brewery_recommender(rand_user).items():
        sim_beers = 0
        for i in score:
            if i < 0.5:
                sim_beers += 1
        
#         if n_beers > 100:
# #             print('sim_beers: ', sim_beers)
#             if int(n_beers * 0.02) <= sim_beers: # >= 2:  not sure exactly how to tune the number of beers at a brewery
#                                              # probably should at least always be greater than 2
#                 best_breweries[brewery] = (np.mean(score), 'number of similar beers: ', sim_beers)
#         else:
        if 3 <= sim_beers:
            if np.mean(score) < 0.5:
                best_breweries[brewery] = (np.mean(score), 'number of similar beers: ', sim_beers)
            else:
                continue
        
    sorted_best_breweries = (sorted(best_breweries.items(), key = 
                 lambda kv:(kv[1], kv[0])))
    if len(sorted_best_breweries) > 3:
        sorted_best_breweries = sorted_best_breweries[:3]
    
    return sorted_best_breweries

In [53]:
%%time
sorting_brewery_scores(4)

CPU times: user 673 ms, sys: 10.1 ms, total: 683 ms
Wall time: 686 ms


[('Brouwerij Van Steenberge N.V.',
  (0.3648140626151446, 'number of similar beers: ', 3))]

In [54]:
%%time
sorting_brewery_scores(20)

CPU times: user 899 ms, sys: 6.79 ms, total: 906 ms
Wall time: 907 ms


[('Brouwerij Van Steenberge N.V.',
  (0.3648140626151446, 'number of similar beers: ', 3))]

In [55]:
%%time
sorting_brewery_scores(100)

CPU times: user 5.86 s, sys: 961 ms, total: 6.82 s
Wall time: 7.03 s


[('Brouwerij Van Steenberge N.V.',
  (0.3648140626151446, 'number of similar beers: ', 3)),
 ('Broadway Pub', (0.3750465504799652, 'number of similar beers: ', 3)),
 ('Brasseurs Illimités',
  (0.41969671505205225, 'number of similar beers: ', 3))]

In [56]:
%%time
sorting_brewery_scores(500)

CPU times: user 25.1 s, sys: 8.67 s, total: 33.8 s
Wall time: 35.6 s


[('Boston Beer Company (Samuel Adams)',
  (0.28268374182401634, 'number of similar beers: ', 4)),
 ('Rock Bottom Restaurant & Brewery',
  (0.3204850690752836, 'number of similar beers: ', 9)),
 ('Bison Brewing', (0.3230617838220723, 'number of similar beers: ', 3))]

In [57]:
recommender_df = []

In [242]:
beers = pivot.sample(n=20, random_state=2) 
np.random.seed(6)
ratings = []
rating = {}
for i in beers.index:
    rating[i] = np.random.uniform(high=5.0)
ratings.append(rating)
pd.DataFrame(user_2_ratings)

,"""100""",'t Smisje BBBourgondier,'t Smisje Blond,(512) Black IPA,(512) Wit,(s)Cream Ale,10 Commandments - Bourbon Barrel-Aged,110K+OT Batch #2 - I.R.I.S.,15th Anniversary Belgian Quadruple,15th Anniversary Wood Aged,...,Zonnestraal,Zubr Classic,Zubr Gold,Zwarte Piet,Zywiec Beer,Åbro Lejon,Åbro Sigill,Érable Rouge,Ölands Amarillo,Ølfabrikken Jule Ale
0,0.319666,0.028048,4.615863,3.546378,0.61343,3.901826,1.060278,3.318456,1.257238,0.573715,...,4.099291,4.94254,1.380904,2.799333,1.394175,3.401256,1.991865,4.783099,4.303194,2.855551


In [243]:
sorting_brewery_scores(user_2_ratings)

[('Southside Speakeasy And Brewpub',
  (0.007722123286332261, 'number of similar beers: ', 2)),
 ('De 7de Hemel', (0.06916307739639443, 'number of similar beers: ', 2)),
 ('Old European Brewing Company',
  (0.07973875349001265, 'number of similar beers: ', 10)),
 ('Roughneck Brewing', (0.10631846792290373, 'number of similar beers: ', 2)),
 ('Birrificio Troll', (0.1369557596364238, 'number of similar beers: ', 2)),
 ('Beachwood BBQ & Brewing',
  (0.13837197982198338, 'number of similar beers: ', 3)),
 ('Taps Fish House & Brewery',
  (0.1684315853500763, 'number of similar beers: ', 2)),
 ('Flour City Brewing Co.',
  (0.174296630661285, 'number of similar beers: ', 3)),
 ('Dockside Restaurant & Brewing Co.',
  (0.18026508780580633, 'number of similar beers: ', 6)),
 ('Darwin Brewery Ltd.',
  (0.18420790327356057, 'number of similar beers: ', 2)),
 ('Greene King / Morland Brewery',
  (0.18590842158930565, 'number of similar beers: ', 2)),
 ('Lotsolox Brewing Company',
  (0.18880332850019

In [37]:
recommender_df['Fat Tire Amber Ale'].sort_values()[1:11]

beer_name
1554 Enlightened Black Ale                    0.538463
Samuel Adams Boston Lager                     0.596796
Sierra Nevada Pale Ale                        0.604438
Sierra Nevada Celebration Ale                 0.615990
Samuel Adams Winter Lager                     0.625337
Samuel Adams Octoberfest                      0.626181
Abbey Belgian Style Ale                       0.627142
Trippel Belgian Style Ale                     0.630708
Hop Rod Rye                                   0.636546
Sierra Nevada Bigfoot Barleywine Style Ale    0.644586
Name: Fat Tire Amber Ale, dtype: float64

# Scraping ratebeer

In [2]:
# scraping for dc breweries
url = 'https://www.ratebeer.com/breweries/washington-dc/48/213/'
response = requests.get(url)
html = response.text
soup = BeautifulSoup(html, 'lxml')

In [4]:
dc_breweries = []
for i in soup.find_all('div', {'class' : 'tab-pane active searchable'}):
    for j in i.find_all('table', {'id' : 'brewerTable'}):
        for k in j.find_all('td'):
            try:
                dc_breweries.append(k.find('a').text)
            except:
                pass

In [7]:
dc_breweries_spaces = []
for i in dc_breweries:
    dc_breweries_spaces.append(i.strip())

In [8]:
dc_breweries_spaces

['3 Stars Brewing Company',
 'Atlas Brew Works',
 'Bardo Brewing',
 'Bluejacket',
 'ChopHouse & Brewery D.C. (Craftworks)',
 'DC Brau Brewing Company',
 'Georgetown Trading Company',
 'Hellbender Brewing Company',
 'Red Bear Brewing Company',
 'Right Proper Brewing Company',
 'Sankofa Beer Company',
 'The Public Option',
 'The Pug',
 'Valor Brewpub']

In [30]:
df[df['brewery_name'].str.contains('Port City')]

,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
459401,24791,Port City Brewing,2012-01-01 01:07:46,3.5,3.0,4.0,brentk56,American Pale Ale (APA),4.0,3.0,Essential Pale Ale,5.5,65927
459402,24791,Port City Brewing,2011-12-17 23:17:13,3.5,3.5,3.5,paulginva,American Pale Ale (APA),3.0,4.0,Essential Pale Ale,5.5,65927
459403,24791,Port City Brewing,2011-12-10 02:53:24,4.0,4.0,3.0,beer2day,American Pale Ale (APA),4.0,4.0,Essential Pale Ale,5.5,65927
459404,24791,Port City Brewing,2011-12-02 23:08:46,2.0,2.0,2.5,wowbagger,American Pale Ale (APA),2.5,3.0,Essential Pale Ale,5.5,65927
459405,24791,Port City Brewing,2011-11-02 02:19:32,3.5,3.0,3.5,LaCoursiere,American Pale Ale (APA),3.5,4.0,Essential Pale Ale,5.5,65927
459406,24791,Port City Brewing,2011-11-01 23:07:56,3.5,3.5,3.5,properhops,American Pale Ale (APA),3.0,4.0,Essential Pale Ale,5.5,65927
459407,24791,Port City Brewing,2011-09-30 20:41:14,5.0,5.0,4.5,Emerson1310,American Pale Ale (APA),4.5,5.0,Essential Pale Ale,5.5,65927
459408,24791,Port City Brewing,2011-08-22 23:00:17,4.0,4.0,4.0,Rhettroactive,American Pale Ale (APA),4.0,4.0,Essential Pale Ale,5.5,65927
459409,24791,Port City Brewing,2011-08-22 11:58:08,3.0,2.5,3.0,BeerFMAndy,American Pale Ale (APA),3.0,3.5,Essential Pale Ale,5.5,65927
459410,24791,Port City Brewing,2011-08-19 20:18:59,3.5,3.5,4.0,jayhawk73,American Pale Ale (APA),3.5,4.0,Essential Pale Ale,5.5,65927
